In [1]:
import os

In [2]:
!nvidia-smi

Thu Feb  1 06:02:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!cp -r '/content/drive/MyDrive/ML projects/brain_data.zip' 'data.zip'

In [4]:
import shutil

In [5]:
shutil.unpack_archive('data.zip', 'brain_data')

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Dropout, GlobalAveragePooling2D, MaxPooling2D, Input, Activation
from tensorflow.keras.layers import Add, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.callbacks import Callback


class PrintEpochStats(Callback):
    """
    Custom print class that outputs bias and variance during training.
    Add to callbacks.
    """
    def on_epoch_end(self, epoch, logs=None):
        print(f"\nBias: {(0.99-logs['Accuracy'])*100:.4f}%, "
              f"Variance: {abs(logs['Accuracy']-logs['val_Accuracy'])*100:.4f}%")

In [11]:
#currnet val acc = 0.82990
check_path = './drive/MyDrive/ML projects/model_saves/weights_vgg2/cp.ckpt'
checkpoints = ModelCheckpoint(check_path,
                              save_weights_only=True,
                              save_best_only=True,
                              monitor='val_Accuracy',
                              verbose=1,
                              mode='max',
                              initial_value_threshold=0.82990)

In [8]:
stopper = EarlyStopping(monitor='val_Accuracy',
                        mode='max',
                        patience=5,
                        verbose=1,
                        restore_best_weights=True)

In [9]:
bias_prints = PrintEpochStats()

In [10]:
callback_list = [checkpoints, bias_prints, stopper]

In [ ]:
#NOTES
# Custom model - overfitting
#Mistake1 - augmented val and test
# Resnet151 - overfitting
# Resnet50 overfitting
# VGG 16- generalizing
#Regularization at 3% bias Lambda: 0.001

In [12]:
# 96% train, 94% val
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import tensorflow as tf

def make_model(input_shape=(224,224,3), fine_tune_at=None):

  base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  base.trainable = False
  x = GlobalAveragePooling2D()(base.output)
  #kernel regularizer
  # x = Dense(4, activation='softmax', kernel_regularizer=L2(0.001))(x)
  x = Dense(4, activation='softmax')(x)


  model = Model(inputs=base.input, outputs=x)
  return model

def vgg_version2():
  base = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
  for layer in base.layers[:-3]:
    layer.trainable = False

  x = GlobalAveragePooling2D()(base.output)
  x = Dense(4, activation='softmax')(x)

  model = Model(inputs=base.input, outputs=x)
  return model

In [ ]:
def make_vgg16():
  input_

In [13]:
datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

In [14]:
train_gen = datagen.flow_from_directory('brain_data/brain_data/Training',
                                        target_size= (224,224),
                                        color_mode='rgb',
                                        seed=20,
                                        batch_size=128,
                                        class_mode='categorical')

test_gen = datagen.flow_from_directory('brain_data/brain_data/Testing',
                                        target_size= (224,224),
                                        color_mode='rgb',
                                        seed=20,
                                        batch_size=128,
                                        class_mode='categorical')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [25]:
x_model = tf.keras.models.load_model('/content/drive/MyDrive/ML projects/model_saves/xception_model.h5')

In [16]:
#tf.keras.backend.clear_session()
model = vgg_version2()

58889256/58889256 [==============================] - 0s 0us/step


In [19]:
model.compile(optimizer=Adam(0.000085),
              loss = 'categorical_crossentropy',
              metrics=['Accuracy'])

In [ ]:
model.load_weights(check_path)

In [ ]:
model.evaluate(test_gen)

11/11 [==============================] - 32s 1s/step - loss: 0.5100 - Accuracy: 0.8238


[0.5100131034851074, 0.823798656463623]

In [20]:
model.fit(train_gen, epochs=50, validation_data=test_gen, callbacks=callback_list)

Epoch 1/50
45/45 [==============================] - ETA: 0s - loss: 0.0033 - Accuracy: 0.9998
Epoch 1: val_Accuracy did not improve from 0.98093

Bias: -0.9825%, Variance: 1.9657%
45/45 [==============================] - 36s 777ms/step - loss: 0.0033 - Accuracy: 0.9998 - val_loss: 0.0525 - val_Accuracy: 0.9802
Epoch 2/50
45/45 [==============================] - ETA: 0s - loss: 0.0015 - Accuracy: 1.0000
Epoch 2: val_Accuracy improved from 0.98093 to 0.98246, saving model to ./drive/MyDrive/ML projects/model_saves/weights/cp.ckpt

Bias: -1.0000%, Variance: 1.7544%
45/45 [==============================] - 35s 772ms/step - loss: 0.0015 - Accuracy: 1.0000 - val_loss: 0.0553 - val_Accuracy: 0.9825
Epoch 3/50
45/45 [==============================] - ETA: 0s - loss: 0.0010 - Accuracy: 1.0000
Epoch 3: val_Accuracy did not improve from 0.98246

Bias: -1.0000%, Variance: 1.9069%
45/45 [==============================] - 34s 746ms/step - loss: 0.0010 - Accuracy: 1.0000 - val_loss: 0.0517 - val_Accu

In [21]:
model.evaluate(train_gen)

45/45 [==============================] - 28s 605ms/step - loss: 7.1133e-04 - Accuracy: 1.0000


[0.0007113322499208152, 1.0]

In [22]:
model.evaluate(test_gen)

11/11 [==============================] - 6s 524ms/step - loss: 0.0553 - Accuracy: 0.9825


[0.05526696890592575, 0.9824561476707458]

In [24]:
path = '/content/drive/MyDrive/ML projects/model_saves/brain_cancer_vgg16.h5'
model.save(path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
vgg_model = tf.keras.models.load_model(path)

In [28]:
vgg_model.evaluate(test_gen)

11/11 [==============================] - 7s 578ms/step - loss: 0.0553 - Accuracy: 0.9825


[0.055266980081796646, 0.9824561476707458]

In [29]:
vgg_model.evaluate(train_gen)

45/45 [==============================] - 27s 584ms/step - loss: 7.1133e-04 - Accuracy: 1.0000


[0.0007113320752978325, 1.0]